In [4]:
from google.colab import drive
import os
import pandas as pd

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/deep_learning')


# 엑셀 파일 읽기
df = pd.read_excel('질병별 증상.xlsx')
df2 = pd.read_excel('증상별 질병.xlsx')
sick = df['질병명']
diag = df['증상']
diag2 = df2['증상']
sick2 = df2['질병명']
# 2022, 2021, 2020 환자수 평균
man = df[['2022_환자수', '2021_환자수', '2020_환자수']].mean(axis=1).round().astype(int)


# 결과를 저장할 딕셔너리 초기화
result = {}

# diag와 sick를 매핑하여 저장
for diag_text, sick_name in zip(diag, sick):
    symptoms = diag_text.split(',')  # 증상을 쉼표로 분리
    for symptom in symptoms:
        if sick_name not in result:
            result[sick_name] = [symptom.strip()]  # 증상을 리스트에 저장
        else:
            result[sick_name].append(symptom.strip())  # 이미 해당 질병이 있다면 새로운 증상 추가


# result 딕셔너리를 데이터프레임으로 변환
result_df = pd.DataFrame({'질병명': list(result.keys()), '증상': list(result.values())})

# man 리스트에 있는 변수의 수 합계 계산
total_count = sum(man)

# 비율 계산 및 개수 조정
expanded_counts = [round(count / total_count * 150000) for count in man]

# 클래스별 데이터 복제
expanded_data = pd.DataFrame(columns=['질병명', '증상'])
for idx, (disease, symptoms) in enumerate(result.items()):
    count = expanded_counts[idx]
    expanded_data = pd.concat([expanded_data, pd.DataFrame({'질병명': [disease] * count, '증상': [symptoms] * count})])

# 데이터 결합
result_expanded = pd.concat([result_df, expanded_data])


Mounted at /content/drive


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MultiLabelBinarizer

# MultiLabelBinarizer를 사용하여 증상을 이진 벡터로 변환
mlb = MultiLabelBinarizer()
symptoms_binary = mlb.fit_transform(result_expanded['증상'])
symptoms_df = pd.DataFrame(symptoms_binary, columns=mlb.classes_)

# 질병명과 이진 벡터로 변환된 증상을 결합
data = pd.concat([result_expanded['질병명'].reset_index(drop=True), symptoms_df.reset_index(drop=True)], axis=1)

# 훈련 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data.drop('질병명', axis=1), data['질병명'], test_size=0.2, random_state=42)

# 결정 트리 분류기 모델 생성
clf = DecisionTreeClassifier()

# 모델 훈련
clf.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = clf.predict(X_test)

# 모델 성능 평가
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8681168593224291


In [6]:
"""
from sklearn.tree import export_graphviz
import graphviz

# 결정 트리 모델 시각화
dot_data = export_graphviz(clf, out_file=None,
                           feature_names=X_train.columns,
                           class_names=clf.classes_,
                           filled=True, rounded=True,
                           special_characters=True)
graph = graphviz.Source(dot_data)
graph.render("decision_tree")

# 결정 트리 모델 시각화 출력
graph
"""

'\nfrom sklearn.tree import export_graphviz\nimport graphviz\n\n# 결정 트리 모델 시각화\ndot_data = export_graphviz(clf, out_file=None,\n                           feature_names=X_train.columns,\n                           class_names=clf.classes_,\n                           filled=True, rounded=True,\n                           special_characters=True)\ngraph = graphviz.Source(dot_data)\ngraph.render("decision_tree")\n\n# 결정 트리 모델 시각화 출력\ngraph\n'

In [7]:
"""
# 결정 트리 분류기 모델 생성 및 하이퍼파라미터 조정
clf = DecisionTreeClassifier(max_depth=10, min_samples_split=10, min_samples_leaf=5, max_features='auto')

# 모델 훈련
clf.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = clf.predict(X_test)

# 모델 성능 평가
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
"""

'\n# 결정 트리 분류기 모델 생성 및 하이퍼파라미터 조정\nclf = DecisionTreeClassifier(max_depth=10, min_samples_split=10, min_samples_leaf=5, max_features=\'auto\')\n\n# 모델 훈련\nclf.fit(X_train, y_train)\n\n# 테스트 데이터에 대한 예측\ny_pred = clf.predict(X_test)\n\n# 모델 성능 평가\naccuracy = accuracy_score(y_test, y_pred)\nprint("Accuracy:", accuracy)\n'

In [8]:
"""
# 결정 트리 분류기 모델 생성 및 하이퍼파라미터 조정
clf = DecisionTreeClassifier(max_depth=15, min_samples_split=20, min_samples_leaf=10, max_features=None)

# 모델 훈련
clf.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = clf.predict(X_test)

# 모델 성능 평가
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
"""

'\n# 결정 트리 분류기 모델 생성 및 하이퍼파라미터 조정\nclf = DecisionTreeClassifier(max_depth=15, min_samples_split=20, min_samples_leaf=10, max_features=None)\n\n# 모델 훈련\nclf.fit(X_train, y_train)\n\n# 테스트 데이터에 대한 예측\ny_pred = clf.predict(X_test)\n\n# 모델 성능 평가\naccuracy = accuracy_score(y_test, y_pred)\nprint("Accuracy:", accuracy)\n'

In [9]:
"""
# 결정 트리 분류기 모델 생성 및 하이퍼파라미터 조정
clf = DecisionTreeClassifier(max_depth=20, min_samples_split=10, min_samples_leaf=5, max_features=None)

# 모델 훈련
clf.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = clf.predict(X_test)

# 모델 성능 평가
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
"""

'\n# 결정 트리 분류기 모델 생성 및 하이퍼파라미터 조정\nclf = DecisionTreeClassifier(max_depth=20, min_samples_split=10, min_samples_leaf=5, max_features=None)\n\n# 모델 훈련\nclf.fit(X_train, y_train)\n\n# 테스트 데이터에 대한 예측\ny_pred = clf.predict(X_test)\n\n# 모델 성능 평가\naccuracy = accuracy_score(y_test, y_pred)\nprint("Accuracy:", accuracy)\n'

In [10]:
"""
# 결정 트리 분류기 모델 생성 및 하이퍼파라미터 조정
clf = DecisionTreeClassifier(max_depth=25, min_samples_split=5, min_samples_leaf=3, max_features=None)

# 모델 훈련
clf.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = clf.predict(X_test)

# 모델 성능 평가
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
"""

'\n# 결정 트리 분류기 모델 생성 및 하이퍼파라미터 조정\nclf = DecisionTreeClassifier(max_depth=25, min_samples_split=5, min_samples_leaf=3, max_features=None)\n\n# 모델 훈련\nclf.fit(X_train, y_train)\n\n# 테스트 데이터에 대한 예측\ny_pred = clf.predict(X_test)\n\n# 모델 성능 평가\naccuracy = accuracy_score(y_test, y_pred)\nprint("Accuracy:", accuracy)\n'

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# 랜덤 포레스트 분류기 모델 생성
rf_clf = RandomForestClassifier()

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 3, 5]
}

# 그리드 서치를 통한 하이퍼파라미터 최적화
grid_search = GridSearchCV(rf_clf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
best_rf_clf = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

# 테스트 데이터에 대한 예측
y_pred = best_rf_clf.predict(X_test)

# 모델 성능 평가
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy: 0.8681168593224291


In [ ]:
from joblib import dump
from joblib import load

# 모델 저장
dump(rf_clf, 'RandomForestClassifier_model.joblib')